# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [8]:
def fizz_buzz_encode(i):
    if i % 15 == 0: return 3
    if i % 5 == 0: return 2
    if i % 3 == 0: return 1
    else: return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

def helper(i):
    print(fizz_buzz_decode(i, fizz_buzz_encode(i)))
    
for i in range(1, 11):
    helper(i)

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz


我们首先定义模型的输入与输出(训练数据)

In [11]:
import numpy as np
import torch

NUM_DIGITS = 10
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

binary_encode(33, NUM_DIGITS)

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2**NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2**NUM_DIGITS)])

In [15]:
trX.shape

torch.Size([923, 10])

然后我们用PyTorch定义模型

In [16]:
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [18]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

以下是模型的训练代码

In [23]:
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]
        
        y_pred = model(batchX)
#         print(y_pred.shape)
        loss = loss_fn(y_pred, batchY)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss = loss_fn(model(trX), trY).item()
    print("Epoch: ", epoch, "Loss: ", loss)

Epoch:  0 Loss:  1.1516852378845215
Epoch:  1 Loss:  1.1488171815872192
Epoch:  2 Loss:  1.1473215818405151
Epoch:  3 Loss:  1.1463618278503418
Epoch:  4 Loss:  1.145654559135437
Epoch:  5 Loss:  1.1450806856155396
Epoch:  6 Loss:  1.1445884704589844
Epoch:  7 Loss:  1.144150972366333
Epoch:  8 Loss:  1.143749475479126
Epoch:  9 Loss:  1.1433781385421753
Epoch:  10 Loss:  1.143028974533081
Epoch:  11 Loss:  1.1427001953125
Epoch:  12 Loss:  1.1423872709274292
Epoch:  13 Loss:  1.142091989517212
Epoch:  14 Loss:  1.1418095827102661
Epoch:  15 Loss:  1.1415385007858276
Epoch:  16 Loss:  1.1412811279296875
Epoch:  17 Loss:  1.141036033630371
Epoch:  18 Loss:  1.1408025026321411
Epoch:  19 Loss:  1.1405764818191528
Epoch:  20 Loss:  1.1403605937957764
Epoch:  21 Loss:  1.1401522159576416
Epoch:  22 Loss:  1.1399517059326172
Epoch:  23 Loss:  1.1397583484649658
Epoch:  24 Loss:  1.1395736932754517
Epoch:  25 Loss:  1.1393942832946777
Epoch:  26 Loss:  1.1392184495925903
Epoch:  27 Loss:  1.

Epoch:  220 Loss:  1.099603295326233
Epoch:  221 Loss:  1.0999325513839722
Epoch:  222 Loss:  1.0999958515167236
Epoch:  223 Loss:  1.0984432697296143
Epoch:  224 Loss:  1.0983080863952637
Epoch:  225 Loss:  1.0978612899780273
Epoch:  226 Loss:  1.0976848602294922
Epoch:  227 Loss:  1.096981406211853
Epoch:  228 Loss:  1.0970649719238281
Epoch:  229 Loss:  1.0962073802947998
Epoch:  230 Loss:  1.0962603092193604
Epoch:  231 Loss:  1.095231294631958
Epoch:  232 Loss:  1.0956180095672607
Epoch:  233 Loss:  1.0952050685882568
Epoch:  234 Loss:  1.0941168069839478
Epoch:  235 Loss:  1.093752145767212
Epoch:  236 Loss:  1.0942234992980957
Epoch:  237 Loss:  1.09260094165802
Epoch:  238 Loss:  1.0932929515838623
Epoch:  239 Loss:  1.0920145511627197
Epoch:  240 Loss:  1.09269118309021
Epoch:  241 Loss:  1.0910706520080566
Epoch:  242 Loss:  1.0905091762542725
Epoch:  243 Loss:  1.09119713306427
Epoch:  244 Loss:  1.090531587600708
Epoch:  245 Loss:  1.0895423889160156
Epoch:  246 Loss:  1.09

Epoch:  450 Loss:  0.9777243733406067
Epoch:  451 Loss:  0.9775205850601196
Epoch:  452 Loss:  0.9765771627426147
Epoch:  453 Loss:  0.97617107629776
Epoch:  454 Loss:  0.9751166105270386
Epoch:  455 Loss:  0.9742923378944397
Epoch:  456 Loss:  0.9740196466445923
Epoch:  457 Loss:  0.9729453325271606
Epoch:  458 Loss:  0.9721157550811768
Epoch:  459 Loss:  0.9717617034912109
Epoch:  460 Loss:  0.9707724452018738
Epoch:  461 Loss:  0.970556914806366
Epoch:  462 Loss:  0.9691885709762573
Epoch:  463 Loss:  0.968742847442627
Epoch:  464 Loss:  0.9686153531074524
Epoch:  465 Loss:  0.9669981598854065
Epoch:  466 Loss:  0.9666933417320251
Epoch:  467 Loss:  0.966088056564331
Epoch:  468 Loss:  0.9652204513549805
Epoch:  469 Loss:  0.9648062586784363
Epoch:  470 Loss:  0.9635184407234192
Epoch:  471 Loss:  0.9635190367698669
Epoch:  472 Loss:  0.9624603986740112
Epoch:  473 Loss:  0.9616624116897583
Epoch:  474 Loss:  0.9615659713745117
Epoch:  475 Loss:  0.9600124955177307
Epoch:  476 Loss:

Epoch:  680 Loss:  0.7767360210418701
Epoch:  681 Loss:  0.7760753631591797
Epoch:  682 Loss:  0.7735989689826965
Epoch:  683 Loss:  0.7738550305366516
Epoch:  684 Loss:  0.7730744481086731
Epoch:  685 Loss:  0.7719939947128296
Epoch:  686 Loss:  0.7713398933410645
Epoch:  687 Loss:  0.770410418510437
Epoch:  688 Loss:  0.7676030397415161
Epoch:  689 Loss:  0.7680120468139648
Epoch:  690 Loss:  0.7677509188652039
Epoch:  691 Loss:  0.766292929649353
Epoch:  692 Loss:  0.7660050392150879
Epoch:  693 Loss:  0.764282763004303
Epoch:  694 Loss:  0.7634891271591187
Epoch:  695 Loss:  0.7628551125526428
Epoch:  696 Loss:  0.7613407969474792
Epoch:  697 Loss:  0.7615928053855896
Epoch:  698 Loss:  0.7579450011253357
Epoch:  699 Loss:  0.7586650848388672
Epoch:  700 Loss:  0.758080005645752
Epoch:  701 Loss:  0.7554884552955627
Epoch:  702 Loss:  0.7536519169807434
Epoch:  703 Loss:  0.7544125318527222
Epoch:  704 Loss:  0.7529691457748413
Epoch:  705 Loss:  0.7522691488265991
Epoch:  706 Loss

Epoch:  909 Loss:  0.5493077635765076
Epoch:  910 Loss:  0.5489935874938965
Epoch:  911 Loss:  0.547457218170166
Epoch:  912 Loss:  0.5466732382774353
Epoch:  913 Loss:  0.5462556481361389
Epoch:  914 Loss:  0.5453401803970337
Epoch:  915 Loss:  0.5439350008964539
Epoch:  916 Loss:  0.5439619421958923
Epoch:  917 Loss:  0.5420883297920227
Epoch:  918 Loss:  0.542199432849884
Epoch:  919 Loss:  0.540179967880249
Epoch:  920 Loss:  0.5405248403549194
Epoch:  921 Loss:  0.5388782024383545
Epoch:  922 Loss:  0.5386988520622253
Epoch:  923 Loss:  0.5369397401809692
Epoch:  924 Loss:  0.5365980863571167
Epoch:  925 Loss:  0.5353963375091553
Epoch:  926 Loss:  0.5343230962753296
Epoch:  927 Loss:  0.5339987874031067
Epoch:  928 Loss:  0.532853364944458
Epoch:  929 Loss:  0.5318193435668945
Epoch:  930 Loss:  0.5313854813575745
Epoch:  931 Loss:  0.5299630761146545
Epoch:  932 Loss:  0.5297728180885315
Epoch:  933 Loss:  0.5286156535148621
Epoch:  934 Loss:  0.5276001691818237
Epoch:  935 Loss

...

Epoch:  9101 Loss:  0.006719778757542372
Epoch:  9102 Loss:  0.006719523575156927
Epoch:  9103 Loss:  0.0067186555825173855
Epoch:  9104 Loss:  0.006716568488627672
Epoch:  9105 Loss:  0.006715908646583557
Epoch:  9106 Loss:  0.006714758928865194
Epoch:  9107 Loss:  0.006713093724101782
Epoch:  9108 Loss:  0.006712213158607483
Epoch:  9109 Loss:  0.006711360067129135
Epoch:  9110 Loss:  0.006710439920425415
Epoch:  9111 Loss:  0.006708710454404354
Epoch:  9112 Loss:  0.006707174237817526
Epoch:  9113 Loss:  0.006706862710416317
Epoch:  9114 Loss:  0.006704933941364288
Epoch:  9115 Loss:  0.006704176310449839
Epoch:  9116 Loss:  0.006703210528939962
Epoch:  9117 Loss:  0.006701409816741943
Epoch:  9118 Loss:  0.006700386758893728
Epoch:  9119 Loss:  0.006699108052998781
Epoch:  9120 Loss:  0.006698540411889553
Epoch:  9121 Loss:  0.006697333883494139
Epoch:  9122 Loss:  0.006695540621876717
Epoch:  9123 Loss:  0.006694474257528782
Epoch:  9124 Loss:  0.006694097071886063
Epoch:  9125 Lo

Epoch:  9307 Loss:  0.006486521102488041
Epoch:  9308 Loss:  0.006485565099865198
Epoch:  9309 Loss:  0.006484621204435825
Epoch:  9310 Loss:  0.006483367178589106
Epoch:  9311 Loss:  0.006482260767370462
Epoch:  9312 Loss:  0.00648093456402421
Epoch:  9313 Loss:  0.006481247954070568
Epoch:  9314 Loss:  0.006479406729340553
Epoch:  9315 Loss:  0.006477902643382549
Epoch:  9316 Loss:  0.006476573180407286
Epoch:  9317 Loss:  0.0064755086787045
Epoch:  9318 Loss:  0.006474860478192568
Epoch:  9319 Loss:  0.006473081652075052
Epoch:  9320 Loss:  0.006472729612141848
Epoch:  9321 Loss:  0.006471227388828993
Epoch:  9322 Loss:  0.006470013875514269
Epoch:  9323 Loss:  0.006468782667070627
Epoch:  9324 Loss:  0.00646777730435133
Epoch:  9325 Loss:  0.006466415710747242
Epoch:  9326 Loss:  0.0064662364311516285
Epoch:  9327 Loss:  0.006464840844273567
Epoch:  9328 Loss:  0.00646372651681304
Epoch:  9329 Loss:  0.006462564691901207
Epoch:  9330 Loss:  0.006460968870669603
Epoch:  9331 Loss:  

Epoch:  9518 Loss:  0.006261640228331089
Epoch:  9519 Loss:  0.006260113324970007
Epoch:  9520 Loss:  0.006258681882172823
Epoch:  9521 Loss:  0.006257679313421249
Epoch:  9522 Loss:  0.006256512366235256
Epoch:  9523 Loss:  0.00625585624948144
Epoch:  9524 Loss:  0.006255059968680143
Epoch:  9525 Loss:  0.00625374261289835
Epoch:  9526 Loss:  0.006252211984246969
Epoch:  9527 Loss:  0.006251018028706312
Epoch:  9528 Loss:  0.006250876002013683
Epoch:  9529 Loss:  0.0062500229105353355
Epoch:  9530 Loss:  0.006248127669095993
Epoch:  9531 Loss:  0.006247965153306723
Epoch:  9532 Loss:  0.006246477365493774
Epoch:  9533 Loss:  0.006244905758649111
Epoch:  9534 Loss:  0.00624442333355546
Epoch:  9535 Loss:  0.006242880132049322
Epoch:  9536 Loss:  0.00624218862503767
Epoch:  9537 Loss:  0.006240909453481436
Epoch:  9538 Loss:  0.006239922251552343
Epoch:  9539 Loss:  0.006239395588636398
Epoch:  9540 Loss:  0.006238445173949003
Epoch:  9541 Loss:  0.006236901972442865
Epoch:  9542 Loss: 

Epoch:  9744 Loss:  0.006033670157194138
Epoch:  9745 Loss:  0.006032919976860285
Epoch:  9746 Loss:  0.006031752098351717
Epoch:  9747 Loss:  0.006030638236552477
Epoch:  9748 Loss:  0.006029867101460695
Epoch:  9749 Loss:  0.006029457785189152
Epoch:  9750 Loss:  0.006028534844517708
Epoch:  9751 Loss:  0.006026489194482565
Epoch:  9752 Loss:  0.006025757174938917
Epoch:  9753 Loss:  0.006024563685059547
Epoch:  9754 Loss:  0.006024018861353397
Epoch:  9755 Loss:  0.006022938527166843
Epoch:  9756 Loss:  0.006021913606673479
Epoch:  9757 Loss:  0.006021516397595406
Epoch:  9758 Loss:  0.006020073778927326
Epoch:  9759 Loss:  0.006018907763063908
Epoch:  9760 Loss:  0.006018489599227905
Epoch:  9761 Loss:  0.006017204374074936
Epoch:  9762 Loss:  0.006016321014612913
Epoch:  9763 Loss:  0.006015060003846884
Epoch:  9764 Loss:  0.006014568265527487
Epoch:  9765 Loss:  0.0060133724473416805
Epoch:  9766 Loss:  0.006012158002704382
Epoch:  9767 Loss:  0.006011088378727436
Epoch:  9768 Lo

Epoch:  9970 Loss:  0.005821219179779291
Epoch:  9971 Loss:  0.0058204601518809795
Epoch:  9972 Loss:  0.005819796584546566
Epoch:  9973 Loss:  0.00581916281953454
Epoch:  9974 Loss:  0.0058175353333354
Epoch:  9975 Loss:  0.005816694349050522
Epoch:  9976 Loss:  0.005815892014652491
Epoch:  9977 Loss:  0.005815385840833187
Epoch:  9978 Loss:  0.005814330652356148
Epoch:  9979 Loss:  0.005813008174300194
Epoch:  9980 Loss:  0.00581216998398304
Epoch:  9981 Loss:  0.005811163689941168
Epoch:  9982 Loss:  0.005810448434203863
Epoch:  9983 Loss:  0.0058097378350794315
Epoch:  9984 Loss:  0.005808277055621147
Epoch:  9985 Loss:  0.005808610003441572
Epoch:  9986 Loss:  0.005806921981275082
Epoch:  9987 Loss:  0.0058057960122823715
Epoch:  9988 Loss:  0.005804968532174826
Epoch:  9989 Loss:  0.005803991574794054
Epoch:  9990 Loss:  0.005802854895591736
Epoch:  9991 Loss:  0.005802706349641085
Epoch:  9992 Loss:  0.005801438819617033
Epoch:  9993 Loss:  0.00580079760402441
Epoch:  9994 Loss:

最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [26]:
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
prediction = zip(range(1,101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for i, x in prediction])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', '21', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', '50', 'fizz', '52', '53', 'fizz', '55', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', '80', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']
